In [1]:
import os
import sys
import math
sys.path.append(os.path.realpath('../..'))
sys.path.append(os.path.realpath('..'))
import aoc
my_aoc = aoc.AdventOfCode(2019,7)
from solution import IntCodeComputer, run_sequence
from itertools import permutations

In [2]:
# class OpCode():
#     """
#     Class to represent an operation
#     """
#     # parameter modes, more a reminder for me, not sure this will get used
#     parameter_modes = {
#         0: 'position',
#         1: 'immediate'
#     }

#     def __init__(self, op_code, parent):
#         """init method"""
#         # set op_code
#         self.op_code = op_code
#         # set parmeter_count, I'm not sure this should be passed here
#         # since everything else about the function is defined here
#         # so should the parameter_count
#         parameter_counts = [0, 3, 3, 1, 1, 2, 2, 3, 3]
#         self.parameter_count = parameter_counts[self.op_code]
#         # set parent, needed to access program memory
#         self.parent = parent

#         # function map of known operations
#         self.opcode_func_map = {
#             1: self.add_and_store,
#             2: self.multiply_and_store,
#             3: self.input_and_store,
#             4: self.retrieve_and_output,
#             5: self.jump_if_true,
#             6: self.jump_if_false,
#             7: self.is_less_than,
#             8: self.is_equals
#         }
#         self.execute = self.opcode_func_map[self.op_code]

#     def get_param_values(self, param_modes, params):
#         """
#         Method to get param values
#         """
#         # init values
#         values = []
#         # get program from parent
#         program = self.parent.program
#         # walk modes to set param values
#         for idx, mode in enumerate(param_modes):
#             # position mode
#             if mode == 0:
#                 # get value from program
#                 values.append(program[params[idx]])
#             # immediate
#             elif mode == 1:
#                 # set value from param
#                 values.append(params[idx])
#             else:
#                 # we shouldn't see this
#                 raise IndexError(f'Invalid parameter mode: {mode}')
#         # return value list
#         return values

#     def add_and_store(self, param_modes, params):
#         """
#         Opcode 1 adds together numbers read from two positions and stores the result in a third
#         position. The three integers immediately after the opcode tell you these three positions
#         - the first two indicate the positions from which you should read the input values, and
#         the third indicates the position at which the output should be stored.
#         """
#         program = self.parent.program
#         values = self.get_param_values(param_modes, params)
#         program[params[2]] = sum(values[:2])

#     def multiply_and_store(self, param_modes, params):
#         """
#         Opcode 2 works exactly like opcode 1, except it multiplies the two inputs instead of
#         adding them. Again, the three integers after the opcode indicate where the inputs and
#         outputs are, not their values.
#         """
#         program = self.parent.program
#         values = self.get_param_values(param_modes, params)
#         program[params[2]] = math.prod(values[:2])

#     def input_and_store(self, param_modes, params):
#         """
#         Opcode 3 takes a single integer as input and saves it to the position
#         given by its only parameter.
#         """
#         # useless action to keep pylint happy
#         type(param_modes)
#         program = self.parent.program
#         # print(f"input_and_store: {self.parent.inputs}")
#         program[params[0]] = self.parent.inputs.pop(0)

#     def retrieve_and_output(self, param_modes, params):
#         """
#         Opcode 4 outputs the value of its only parameter.
#         """
#         # useless action to keep pylint happy
#         type(param_modes)
#         parent = self.parent
#         program = parent.program
#         parent.last_output = program[params[0]]
#         parent.output.append(program[params[0]])
#         return program[params[0]]

#     def jump_if_true(self, param_modes, params):
#         """
#         Opcode 5 is jump-if-true: if the first parameter is non-zero, it sets the instruction
#         pointer to the value from the second parameter. Otherwise, it does nothing.
#         """
#         values = self.get_param_values(param_modes, params)
#         if values[0]:
#             # print(True, values[1])
#             self.parent.jump = values[1]

#     def jump_if_false(self, param_modes, params):
#         """
#         Opcode 6 is jump-if-false: if the first parameter is zero, it sets the instruction
#         pointer to the value from the second parameter. Otherwise, it does nothing.
#         """
#         values = self.get_param_values(param_modes, params)
#         if not values[0]:
#             self.parent.jump = values[1]

#     def is_less_than(self, param_modes, params):
#         """
#         Opcode 7 is less than: if the first parameter is less than the second parameter,
#         it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
#         """
#         program = self.parent.program
#         values = self.get_param_values(param_modes, params)
#         if values[0] < values[1]:
#             program[params[2]] = 1
#         else:
#             program[params[2]] = 0

#     def is_equals(self, param_modes, params):
#         """
#         Opcode 8 is equals: if the first parameter is equal to the second parameter,
#         it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
#         """
#         program = self.parent.program
#         values = self.get_param_values(param_modes, params)
#         if values[0] == values[1]:
#             program[params[2]] = 1
#         else:
#             program[params[2]] = 0

# class IntCodeComputer():
#     """
#     Class to simulate an IntCode Computer
#     """
#     def __init__(self, icc_id, program, input_val=None):
#         """init method"""
#         # set id
#         self.icc_id = icc_id
#         # parse program
#         self.program = [int(num) for num in program.split(',')]
#         # init pointer
#         self.ptr = 0
#         # init running
#         self.running = True
#         # init input
#         self.inputs = []
#         if input_val is not None:
#             self.inputs.append(input_val)
#         self.output = None
#         self.last_output = None
#         # init operations
#         self.operations = {}
#         # define operations
#         for op_code in range(1, 8 + 1):
#             self.operations[op_code] = OpCode(op_code, self)
#         # init jump
#         self.jump = None

#     def set_output(self, other):
#         """Method to connect output to input"""
#         self.output = other.inputs

#     def run(self, input_val=None):
#         """
#         Method to run program
#         """
#         # set input if passed
#         if not input_val is None:
#             self.inputs.append(input_val)
#         # init ptr
#         self.ptr = 0
#         # init last_resut
#         last_output = None
#         # loop until break
#         while True:
#             # run next step
#             output = self.step()
#             # check for result
#             if not output is None:
#                 # check for break(99)
#                 if output == 99:
#                     break
#                 # save output
#                 last_output = output
#         # return last output
#         return last_output

#     def step(self):
#         """
#         Method to step into program one step
#         """
#         # get current intstruction string
#         instruction_text = str(self.program[self.ptr])
#         # the opcode is the rightmost two digits of the first value in an instruction.
#         op_code = int(instruction_text[-2:])
#         if op_code == 99:
#             self.running = False
#             #99 means that the program is finished and should immediately halt
#             return 99
#         # waiting on input
#         if op_code == 3 and len(self.inputs) == 0:
#             # return without moving ptr
#             return -1
#         # get operation
#         try:
#             operation = self.operations[op_code]
#         except KeyError:
#             print(f"Invalid op_code {op_code} at ptr {self.ptr}")
#             return 99
#         # Parameter modes are single digits, one per parameter, read right-to-left from the opcode
#         param_modes = list(reversed([int(num) for num in instruction_text[:-2]]))
#         # Any missing modes are 0
#         while len(param_modes) < operation.parameter_count:
#             param_modes.append(0)
#         # init params
#         params = []
#         # get a param for each parameter_count
#         for _ in range(operation.parameter_count):
#             # increment ptr
#             self.ptr += 1
#             # get next param
#             params.append(self.program[self.ptr])
#         # execute operation
#         result = operation.execute(param_modes, params)
#         # if operation set jump, then we jump
#         if self.jump:
#             # move ptr to jump value
#             self.ptr = self.jump
#             # clear jump value
#             self.jump = None
#         else:
#             # increment ptr to next instruction
#             self.ptr += 1
#         return result


In [3]:
test_cases = []
test_cases.append(
    { 
        "program": "3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0",
        "sequence": [4, 3, 2, 1, 0],
        "expected_result": 43210
    }
)
test_cases.append(
    { 
        "program": "3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0",
        "sequence": [0, 1, 2, 3, 4],
        "expected_result": 54321
    }
)
test_cases.append(
    { 
        "program": "3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0",
        "sequence": [1, 0, 4, 3, 2],
        "expected_result": 65210
    }
)
for idx, test_case in enumerate(test_cases):
    # inint max_output
    max_output = 0
    # iterate over possible sequences
    for sequence in permutations(range(5)):
        # init output
        output = run_sequence(program=test_case['program'], sequence=test_case['sequence'])
        if output is not None:
            # check final output againse max_output
            max_output = max(max_output, output)
    # return largest value
    
    # print(f"Looking for {test_case['expected_result']}:")
    # for amp_id, amp in amps.items():
    #     print(amp_id, amp.last_output)
    # print()
    assert max_output == test_case['expected_result']
    print(idx, max_output)


0 43210
1 54321
2 65210


In [4]:
test_cases = []
test_cases.append(
    { 
        "program": "3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5",
        "sequence": [9, 8, 7, 6, 5],
        "expected_result": 139629729
    }
)
test_cases.append(
    { 
        "program": "3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10",
        "sequence": [9, 7 ,8 ,5 ,6],
        "expected_result": 18216
    }
)
for test_case in test_cases:
    max_output = 0
    for sequence in permutations(range(5, 10)):
        output = run_sequence(test_case['program'], sequence)
        max_output = max(max_output, output)
    print(max_output)
    assert max_output == test_case['expected_result']
    


139629729
18216
